# Import the data frames

Sorted out the columns - confirmed they are all there.  
Now need to drop unnecessary columns as I go

In [252]:
import pandas as pd
import numpy as np
import seaborn as sns

In [253]:
drug = pandas.read_csv("T201911CHEM_SUBS.csv")
gp = pandas.read_csv("gpheadings.csv")
people = pandas.read_csv("people.csv")
structure = pandas.read_csv("structure.csv")
allgp = pandas.read_csv("allgp.csv")
toypres = pandas.read_csv("ToyPres.csv")

# Functions

In [254]:
# Rename any column that ends in _x from the merger function
def rename_x(df):
    for col in df:
        if col.endswith('_x'):
            df.rename(columns={col:col.rstrip('_x')}, inplace=True)


In [255]:
# Remove any column that is unnamed in the dataframe 
# Bit of a misnomer
def rename_unname(df):
    for col in df:
        if col.startswith('Unnamed'):
            df.drop(col,axis=1, inplace=True)

In [256]:
# Drop any column names _y at the end, as this is a duplicate column from the merge unc
def drop_y(df):
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
#Might try and not run this for a bit, as think it might be dropping too much?

# Merging datasets

### gp and allgp

In [257]:
rename_unname(gp)
rename_unname(allgp)
allgp.rename(columns={'Postcode':'To_drop', '1974...':'Postcode', 'NA.10':'Setting_all_gp_reference', 'NA.8':'Provider'}, inplace=True)

In [258]:
#allgp.head()

In [259]:
to_drop = ['To_drop', 'NA', 'NA.1', 'NA.2','NA.3', 'NA.4', 'NA.5', 'NA.6', 'NA.7', 'NA.9', 'NA.11']
allgp.drop(to_drop, axis=1, inplace=True) 

# Remove the unneccessaruy columns

In [260]:
print(list(allgp.columns))

['Organisation_code', 'Address_1', 'National_grouping', 'High_level_health_geography', 'Address_2', 'Address_3', 'Address_4', 'Area', 'Postcode', 'Date_open', 'Date_close', 'Status_code', 'Subtype', 'Commissioner', 'Provider', 'Setting_all_gp_reference']


In [261]:
# Create a dataset that is a merged version of gp and all gp
final = pd.merge(gp, allgp, how="outer", left_on=["E8..."], right_on=["Organisation_code"])

In [262]:
final.drop(['Organisation_code', 'Provider', 'Address_4'], axis=1, inplace=True) 
drop_y(final)
rename_x(final)
rename_unname(final)
final.head()

,timepoint,E8...,Name,Address_1,Address_2,Address_3,Area,Postcode,National_grouping,High_level_health_geography,Date_open,Date_close,Status_code,Subtype,Commissioner,Setting_all_gp_reference
0,201911.0,E82603,CASSIO SURGERY,1ST FLOOR COLNE HOUSE,21 UPTON ROAD,WATFORD,HERTFORDSHIRE,WD18 0JP,Y61,Q79,19740401,NaN,A,B,06N,4
1,201911.0,E82641,MANOR HOUSE SURGERY,MANOR HOUSE SURGERY,EMPERORS GATE,CHELLS MANOR STEVENAGE,HERTFORDSHIRE,SG2 7QX,Y61,Q79,19890220,NaN,A,B,06K,4
2,201911.0,E83039,RAVENSCROFT MEDICAL CENTRE,RAVENSCROFT MEDICAL CTR,166-168 GOLDERS GREEN RD,GOLDERS GREEN,LONDON,NW11 8BB,Y56,Q71,19740401,NaN,A,B,07M,4
3,201911.0,E83046,MULBERRY MEDICAL PRACTICE,3 SEFTON AVENUE,NaN,MILL HILL,LONDON,NW7 3QB,Y56,Q71,19740401,NaN,A,B,07M,4
4,201911.0,E84020,JAI MEDICAL CENTRE (BRENT),82 STAG LANE,EDGWARE,MIDDLESEX,NaN,HA8 5LP,Y56,Q71,19740401,NaN,A,B,07P,4


### final & people

In [263]:
rename_unname(people)
people.head()

,PUBLICATION,EXTRACT_DATE,TYPE,CCG_CODE,ONS_CCG_CODE,CODE,POSTCODE,SEX,AGE,NUMBER_OF_PATIENTS
0,GP_PRAC_PAT_LIST,01NOV2019,GP,00C,E38000042,A83005,DL1 3RT,ALL,ALL,11999
1,GP_PRAC_PAT_LIST,01NOV2019,GP,00C,E38000042,A83006,DL3 6HZ,ALL,ALL,8093
2,GP_PRAC_PAT_LIST,01NOV2019,GP,00C,E38000042,A83010,DL3 9JP,ALL,ALL,13749
3,GP_PRAC_PAT_LIST,01NOV2019,GP,00C,E38000042,A83013,DL1 4YL,ALL,ALL,11740
4,GP_PRAC_PAT_LIST,01NOV2019,GP,00C,E38000042,A83031,DL3 8SQ,ALL,ALL,10441


In [264]:
final = pd.merge(final, people, how='outer', left_on=['E8...'], right_on=['CODE'])
final.drop(['PUBLICATION', 'EXTRACT_DATE', 'CODE', 'POSTCODE'], axis=1, inplace=True)
print(list(final.columns) )
print('-'*20)
print(final.shape)

['timepoint', 'E8...', 'Name', 'Address_1', 'Address_2', 'Address_3', 'Area', 'Postcode', 'National_grouping', 'High_level_health_geography', 'Date_open', 'Date_close', 'Status_code', 'Subtype', 'Commissioner', 'Setting_all_gp_reference', 'TYPE', 'CCG_CODE', 'ONS_CCG_CODE', 'SEX', 'AGE', 'NUMBER_OF_PATIENTS']
--------------------
(14388, 22)


In [265]:
#final.head()

### final & structure

In [266]:
rename_unname(structure)
structure.head()

,Organisation_code,CCG/PCT,Primary_care_organisation_type,Join_parent_date,Left_parent_date,Amended_record_indicator
0,A81001,00K,W,20130401,NaN,0
1,A81001,4QP36,W,19990401,20010331.0,0
2,A81002,5E1,W,20010401,20130331.0,0
3,A81002,4QP36,W,19990401,20010331.0,0
4,A81002,00K,W,20130401,NaN,0


In [267]:
final = pd.merge(final, structure, how="outer", left_on=["E8..."], right_on=["Organisation_code"])
print(list(final.columns) )
print('-'*20)
print(final.shape)

['timepoint', 'E8...', 'Name', 'Address_1', 'Address_2', 'Address_3', 'Area', 'Postcode', 'National_grouping', 'High_level_health_geography', 'Date_open', 'Date_close', 'Status_code', 'Subtype', 'Commissioner', 'Setting_all_gp_reference', 'TYPE', 'CCG_CODE', 'ONS_CCG_CODE', 'SEX', 'AGE', 'NUMBER_OF_PATIENTS', 'Organisation_code', 'CCG/PCT', 'Primary_care_organisation_type', 'Join_parent_date', 'Left_parent_date', 'Amended_record_indicator']
--------------------
(52928, 28)


In [268]:
rename_x(final)
drop_y(final)

### final & toypres 
Going to need to scale this up for the prescription data when running it on the HPC

In [269]:
#toypres.head()

In [270]:
rename_unname(toypres)
toypres.drop(['PCT'], axis=1, inplace=True)
final = pd.merge(final, toypres, how="outer", left_on=["E8..."], right_on=["PRACTICE"])

In [271]:
#final.head(1)

In [272]:
print(list(final.columns) )
print('-'*20)
print(final.shape)

['timepoint', 'E8...', 'Name', 'Address_1', 'Address_2', 'Address_3', 'Area', 'Postcode', 'National_grouping', 'High_level_health_geography', 'Date_open', 'Date_close', 'Status_code', 'Subtype', 'Commissioner', 'Setting_all_gp_reference', 'TYPE', 'CCG_CODE', 'ONS_CCG_CODE', 'SEX', 'AGE', 'NUMBER_OF_PATIENTS', 'Organisation_code', 'CCG/PCT', 'Primary_care_organisation_type', 'Join_parent_date', 'Left_parent_date', 'Amended_record_indicator', 'SHA', 'PRACTICE', 'BNF.CODE', 'BNF.NAME', 'ITEMS', 'NIC', 'ACT.COST', 'QUANTITY', 'PERIOD']
--------------------
(53236, 37)


### Looking at drug - seeing if can join in
Not sure if this is necessary because already have the drug BNF codes

# Saving the resulting csv

In [273]:
import csv
final.to_csv("Combined_TOY_NHS_data.csv")